In [2]:
import gzip
from collections import defaultdict
import string
from sklearn import svm

In [3]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [84]:
train_data = []
valid_data = []
data = []
userCatCounts = defaultdict(list)

words_cat = defaultdict(list)
for l in readGz('train.json.gz'):
    user,category,item,review = l['reviewerID'],l['categoryID'],l['itemID'],l['reviewText']
    data.append((user, category, item, review))
    if user not in userCatCounts:
        userCatCounts[user] = [0, 0, 0, 0, 0]
    if category == 0:
        words_cat[0].append(review)
        userCatCounts[user][0] += 1
    elif category == 1:
        words_cat[1].append(review)
        userCatCounts[user][1] += 1
    elif category == 2:
        words_cat[2].append(review)
        userCatCounts[user][2] += 1
    elif category == 3:
        words_cat[3].append(review)
        userCatCounts[user][3] += 1
    elif category == 4:
        words_cat[4].append(review)
        userCatCounts[user][4] += 1
train_data = data[:155000]
valid_data = data[155000:]
print(len(userCatCounts))

39239


In [85]:
#preprocess the data and find the counts for each word.
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in train_data:
    r = ''.join([c for c in d[3].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

In [86]:
def getTopWords(data):
    #get the 750 most common words
    common = set(['a', 'and', 'but', 'from', 'to', 'on', 'for', 'the', 'of'])
    counts = [(data[w], w) for w in data]
    counts.sort()
    counts.reverse()
    words = [x for x in counts[:1000] if not x in common]
    return words

In [87]:
words = getTopWords(wordCount)
total = 0
for tup in words:
    total += tup[0]
freqWordsTotal = {}
for x in words:
    freqWordsTotal[x[1]] = x[0] / total

In [88]:
catDataDict = defaultdict(list)
for tup in train_data:
    if tup[1] == 0:
        catDataDict[0].append(tup)
    elif tup[1] == 1:
        catDataDict[1].append(tup)
    elif tup[1] == 2:
        catDataDict[2].append(tup)
    elif tup[1] == 3:
        catDataDict[3].append(tup)
    elif tup[1] == 4:
        catDataDict[4].append(tup)
        
        

def computeFreqForCat(data):
    wc = defaultdict(int)
    punctuation = set(string.punctuation)
    for tup in data:
        r = ''.join([c for c in tup[3].lower() if not c in punctuation])
        for w in r.split():
            wc[w] += 1
    return wc

In [89]:
catFreqs = defaultdict(dict)
for i in range(5):
    wc = computeFreqForCat(catDataDict[i])
    total = 0
    new_dict = {}
    for key, val in list(freqWordsTotal.items()):
        total += wc[key]
    for key, val in list(freqWordsTotal.items()):
        new_dict[key] = wc[key] / total
    catFreqs[i] = new_dict

In [90]:
temp_dict ={}
topWords = defaultdict(list)
for i in range(5):
    l = []
    for key, val in list(freqWordsTotal.items()):
        diff = catFreqs[i][key] - val
        l.append((diff, key))
    temp_dict[i] = l
for i in range(5):
    temp_dict[i].sort()
    temp_dict[i].reverse()
    for x in range(700):
        topWords[i].append(temp_dict[i][x])

In [91]:
finalWords = []
for val in topWords.values():
    finalWords.extend(val)

In [93]:
len(set(finalWords))

3499

In [94]:
X = []
y = []
for d in train_data:
    y.append(d[1])
    feat = [x[1] in d[3] for x in set(finalWords)]
    feat.extend(userCatCounts[d[0]])
    X.append(feat)
print(len(X), len(y))

155000 155000


In [95]:
X_valid = []
y_valid = []
for d in valid_data:
    y_valid.append(d[1])
    feat = [x[1] in d[3] for x in set(finalWords)]
    feat.extend(userCatCounts[d[0]])
    X_valid.append(feat)
print(len(X_valid), len(y_valid))

45000 45000


In [96]:
clf = svm.LinearSVC(C=10.0, multi_class='ovr', dual=False)
clf.fit(X, y)

LinearSVC(C=10.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [97]:
clf.score(X, y)

0.9097677419354838

In [98]:
clf.score(X_valid, y_valid)

0.9052

In [104]:
predictions = open("predictions_Category.txt", 'w')
predictions.write("reviewerID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
    x_test = []
    words = l['reviewText'].lower()
    feat = [x[1] in words for x in set(finalWords)]
    counts = userCatCounts[l['reviewerID']]
    if len(counts) == 0:
        counts = [0, 0, 0, 0, 0]
    feat.extend(counts)
    x_test.append(feat)
    pred = clf.predict(x_test)
    predictions.write(l['reviewerID'] + '-' + l['reviewHash'] + "," + str(pred[0]) + "\n")

predictions.close()

PurchasePredictions:

In [119]:
train_data2 = []
valid_data2 = []
users = []
items = []
userCategories = defaultdict(list)
itemCategories = defaultdict(list)
user_item_dict = defaultdict(set)
i = 0
for l in readGz('train.json.gz'):
    user,category,item,rating,helpful,summary  = l['reviewerID'],l['categoryID'],l['itemID'], l['rating'], l['helpful'], l['summary']
    items.append(item)
    users.append(user)
    user_item_dict[user].add(item)
    if i < 100000:
        userCategories[user].append((category, rating, helpful, summary, item))
        itemCategories[item].append((category, rating, helpful, summary, item))
        train_data2.append((user, item))
    else:
        valid_data2.append((user, item))
    i += 1
user_set = set(users)
item_set = set(items)

In [120]:
#pick negative samples for the valid data
import random
num = 0
while num < 100000:
    u = random.sample(user_set, 1)
    i = random.sample(item_set, 1)
    if i[0] not in user_item_dict[u[0]]:
        train_data2.append((u[0], i[0], -1))
        num += 1
        
num = 0
while num < 100000:
    u = random.sample(user_set, 1)
    i = random.sample(item_set, 1)
    if i[0] not in user_item_dict[u[0]]:
        valid_data2.append((u[0], i[0], -1))
        num += 1

In [122]:
#retrain the baseline on just the new train data
businessCount = defaultdict(int)
userCount = defaultdict(int)
totalPurchases = 0

#item pop:
for tup in train_data2:
    item = tup[1]
    user = tup[0]
    businessCount[item] += 1
    userCount[user] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/1.8: break
        
mostPopular = [(userCount[x], x) for x in userCount]
mostPopular.sort()
mostPopular.reverse()

return2 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return2.add(i)
    if count > totalPurchases/1.8: break

In [123]:
import numpy as np

itemAvg = {}
total = 0
#find the popularity of the items
for key, val in list(itemCategories.items()):
    #key is the item and val is the data tuple
    total += len(val)
for key, val in list(itemCategories.items()):
    t = 0
    for tup in val:
        t += tup[1]
    itemAvg[key] = t / len(val)

In [124]:
userAvg = {}
for key, val in list(userCategories.items()):
    t = 0
    for tup in val:
        t += tup[1]
    userAvg[key] = t / len(val)

In [198]:
userCatCounts = defaultdict(dict)
for key, val in list(userCategories.items()):
    for tup in val:
        if tup[0] in userCatCounts[key]:
            userCatCounts[key][tup[0]] += 1
        else:
            userCatCounts[key][tup[0]] = 1

In [262]:
# itemCounts = defaultdict(int)
# for key, val in list(userCategories.items()):
#     for i in range(len(val)):
#         for j in range(len(val)):
#             if i == j: continue
#             itemCounts[(val[i][4], val[j][4])] += 1
#             itemCounts[(val[j][4], val[i][4])] += 1

In [263]:
len(itemCounts)

287782

In [273]:

print(len(y))

200000


In [274]:
y = []
for i in range(100000):
    y.append(1)
for i in range(100000):
    y.append(0)
X = []
for data in train_data2:
    item, user = data[1], data[0]
    feat = []
    if item not in itemAvg or user not in userCategories:
        feat = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        X.append(feat)
        continue
    cat = itemCategories[item][0][0]
    feat.append(cat)
    
    avg = itemAvg[item]
    feat.append(avg)
    
    ua = userAvg[user]
    feat.append(ua)
    
    feat.append(item in return1)
    feat.append(user in return2)
    for i in range(5):
        if i in userCatCounts[user]:
            cnt = userCatCounts[user][i]
            feat.append(cnt)
        else:
            feat.append(0)
    X.append(feat)

200000


In [275]:
y_valid = []
for i in range(100000):
    y_valid.append(1)
for i in range(100000):
    y_valid.append(0)
X_valid = []
for data in valid_data2:
    item, user = data[1], data[0]
    feat = []
    if item not in itemAvg or user not in userCategories:
        feat = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        X_valid.append(feat)
        continue
    feat.append(tc)
    cat = itemCategories[item][0][0]
    
    feat.append(cat)
    
    avg = itemAvg[item]
    feat.append(avg)
    
    ua = userAvg[user]
    feat.append(ua)
    
    feat.append(item in return1)
    feat.append(user in return2)
    for i in range(5):
        if i in userCatCounts[user]:
            cnt = userCatCounts[user][i]
            feat.append(cnt)
        else:
            feat.append(0)
    X_valid.append(feat)

200000 200000 200000 200000


In [288]:
clf = svm.LinearSVC(C=10.0, dual=False)
clf.fit(X, y)
clf.score(X, y)
clf.score(X_valid, y_valid)

LinearSVC(C=10.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

0.956625

0.516465

In [240]:
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    user,item = l.strip().split('-')
    
    X_test = []
    feat = [1]
    if item not in itemAvg or user not in userCategories:
        feat = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        X_test.append(feat)
    else:
        cat = itemCategories[item][0][0]

        feat.append(cat)

        avg = itemAvg[item]
        feat.append(avg)

        ua = userAvg[user]
        feat.append(ua)

        feat.append(item in return1)
        feat.append(user in return2)
        for i in range(5):
            if i in userCatCounts[user]:
                cnt = userCatCounts[user][i]
                feat.append(cnt)
            else:
                feat.append(0)
        X_test.append(feat)
    pred = clf.predict(X_test)
    predictions.write(user + '-' + item + ","+str(pred[0])+"\n")

predictions.close()